In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load the dataset
file_path = '/Users/simgyuseong/Documents/Hate_Detection/dataset/raw/unsmile/unsmile_dataset.txt'
data = pd.read_csv(file_path, delimiter='\t')

# Display the first few rows of the dataset
print(data.head())

# Define the labels
labels = ['여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean', '개인지칭']

# Text cleaning function
def clean_text(text):
    # Remove non-alphanumeric characters and extra spaces
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"[^a-zA-Z가-힣0-9\s]", "", text)
    return text.strip()

# Clean the '문장' column
data['문장'] = data['문장'].apply(clean_text)

# Convert labels to a list of labels
def extract_labels(row):
    return [labels[i] for i in range(len(labels)) if row[i+1] == 1]

data['labels'] = data.apply(extract_labels, axis=1)

# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['문장'])
sequences = tokenizer.texts_to_sequences(data['문장'])

# Pad the sequences
maxlen = 100  # You can adjust the maxlen according to your needs
X = pad_sequences(sequences, maxlen=maxlen)

# Binarize the labels
mlb = MultiLabelBinarizer(classes=labels)
y = mlb.fit_transform(data['labels'])

# Print the shape of the dataset
print(f'Shape of X: {X.shape}')
print(f'Shape of y: {y.shape}')

                                                  문장  여성/가족  남성  성소수자  인종/국적  \
0  ㅇㄱㄹㅇ 진짜 죽어도 상관없다는 마인드로 싸웠더니 지금 서열 상타취노 식칼들고 니가...      0   1     0      0   
1                         여자들은 취미가 애낳는건가.. 취미를 좀 가져라      1   0     0      0   
2                           개슬람녀 다 필요없고 니 엄마만 있으면 된다      0   0     0      1   
3  조팔ㅋㅋ 남한 길거리 돌아다니면 한국남자때문에 눈재기하는데 그걸 내 폰에 굳이 담아...      0   1     0      0   
4                              바지 내리다 한남들 와꾸 보고 올려뿟노      0   1     0      0   

   연령  지역  종교  기타 혐오  악플/욕설  clean  개인지칭  
0   0   0   0      0      0      0     0  
1   0   0   0      0      0      0     0  
2   0   0   1      0      0      0     0  
3   0   0   0      0      0      0     0  
4   0   0   0      0      0      0     0  


/var/folders/sd/pq30895j7xl996rt4n_pcm0r0000gn/T/ipykernel_80817/4203024401.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return [labels[i] for i in range(len(labels)) if row[i+1] == 1]


Shape of X: (18742, 100)
Shape of y: (18742, 11)


In [4]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 79471
등장 빈도가 2번 이하인 희귀 단어의 수: 71542
단어 집합에서 희귀 단어의 비율: 90.02277560367932
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 49.58030465836546


In [5]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 7931
